# *Baseline* and *Mitigation* Models - CNN

In [1]:
!pip install adversarial-robustness-toolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.9 MB/s eta 0:00:0000:01


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier
import numpy as np

In [3]:
# defining model architectures

class SimpleCNN(nn.Module):
    def __init__(self, num_classes = 10): # specifically for MNIST
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim = 1)
        
        return output

In [4]:
# defining training training hyperparameters

EPOCHS_BASELINE = 5
EPOCHS_MITIGATION = 20
BATCH_SIZE = 128
LEARNING_RATE = 0.001
EPSILON = 0.2 # pertuberation size for PGD attack

# checking for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
# loading data

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
train_dataset = datasets.MNIST(root = '/kaggle/working', train = True, download = True, transform = transform)
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 17.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 478kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.47MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.65MB/s]


In [ ]:
# training function

def train_model(model, train_loader, optimizer, loss_fn, epochs, description):
    model.train()
    print(f"Starting {description} training...")
    for epoch in range(1, epochs + 1):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward() 
            optimizer.step()
    
    print(f"{description} training completed")

In [7]:
# training phase 1 - base classification model

baseline_model = SimpleCNN().to(device)
baseline_optimizer = optim.Adam(baseline_model.parameters(), lr = LEARNING_RATE)
train_model(baseline_model, train_loader, baseline_optimizer, F.nll_loss, EPOCHS_BASELINE, "Baseline Model")
torch.save(baseline_model.state_dict(), "baseline_model.pth")
print("Baseline model saved as 'baseline_model.pth'")

Starting Baseline Model training...


/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:1538: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Baseline Model training completed
Baseline model saved as 'baseline_model.pth'


In [14]:
# training phase 2 - mitigation model with adversarial training

mitigation_model = SimpleCNN().to(device)
mitigation_optimizer = optim.Adam(mitigation_model.parameters(), lr = LEARNING_RATE)
mitigation_classifier_art = PyTorchClassifier(model = mitigation_model,
                                              loss = nn.CrossEntropyLoss(),
                                              optimizer = mitigation_optimizer,
                                              input_shape = (1, 28, 28),
                                              nb_classes = 10,
                                              device_type = 'gpu')

# Convert targets to one-hot encoding
y_one_hot = torch.nn.functional.one_hot(torch.from_numpy(train_loader.dataset.targets.numpy()).long(), num_classes = 10).numpy()

mitigation_classifier_art.fit(x = train_loader.dataset.data.numpy().reshape(-1, 1, 28, 28), 
                              y = y_one_hot,
                              batch_size = BATCH_SIZE, nb_epochs = EPOCHS_MITIGATION,
                              attack_params = {
                                  'projected_gradient_descent': {
                                      'eps': EPSILON,
                                      'eps_step': 0.01,
                                      'max_iter': 20}})

torch.save(mitigation_model.state_dict(), "mitigation_model.pth")
print("Mitigation model saved as 'mitigation_model.pth'")

Mitigation model saved as 'mitigation_model.pth'
